<h1><center>Driver Drowsiness Detection</center></h1>


**Importing the libraries for the model building:**
<br>
<br>
**import os**:this module will provide functions for creating and removing a directory (folder), fetching its contents, changing and identifying the current directory, etc. We have to import the os module to interact with the underlying operating system.
<br>
**keras.preprocessing import image**: used for real-time data augmentation on image data.
<br>
**matplotlib.pyplot as plt**: used for generating visualizations
<br>
**import numpy as np**: used for array related functions
<br>
**keras.utils.np_utils import to_categorical**: used to Convert a class vector (integers) to binary class matrix
<br>
**import random**:used for pseudo-random number generators for various distributions
<br>
**import shutil**:provides support for file copying and removal
<br>
**keras.models import Sequential**: Sequential model is used for a plain stack of layers where each layer has one input tensor and one output tensor.
<br>
**Dropout**: is a technique where randomly selected neurons are ignored during training to prevent overfitting
<br>
**Conv2D**:is a 2D Convolution Layer, this layer creates a convolution kernel that is wind with layers input which helps produce a tensor of outputs.
<br>
**Flatten**: it reshapes the tensor to have the shape that is equal to the number of elements contained in tensor non including the batch dimension
<br>
**Dense**: a dense layer is deeply connected with its preceding layer which means the neurons of the layer are connected to every neuron of its preceding layer.
<br>
**MaxPooling2D**: Max pooling selects the brighter pixels from the image. It is used to downsamples the input along its spatial dimensions (height and width) by taking the maximum value over an input window (of size defined by pool_size ) for each channel of the input.
<br>
**BatchNormalization**: it is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch.

In [1]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model

We are defining a function generator to genearte the train and test batch and rescalae and categorize the data. The data we are using here is a already available data set and can be fteched from [here](https://drive.google.com/drive/folders/1nTb-_PURK8XO8yv36MzFoviJQ6csvniL?usp=sharing). The data is divided into train and test.

In [4]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical' ):

    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=target_size)

batch_size= 32
target_size=(24,24)
train_batch= generator('data/train',shuffle=True, batch_size=batch_size,target_size=target_size)
test_batch= generator('data/test',shuffle=True, batch_size=batch_size,target_size=target_size)
steps_per_epoch= len(train_batch.classes)//batch_size
test_steps = len(test_batch.classes)//batch_size
print(f'Steps Per Epoch= {steps_per_epoch}',f'Test Steps= {test_steps}')


Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.
Steps Per Epoch= 38 Test Steps= 6


**Convolutional Neural Network** is a deep neural network which performs extremely well for image classification purposes. A CNN basically consists of an input layer, an output layer and a hidden layer which can have multiple layers. A convolution operation is performed on these layers using a filter that performs 2D matrix multiplication on the layer and filter.

The Sequential CNN model we are going to define will consist of the following layers:

Convolutional layer; 32 neurons, kernel size 3
<br>
Convolutional layer; 32 neurons, kernel size 3
<br>
Convolutional layer; 64 neurons, kernel size 3
<br>
Fully connected layer; 128 neurons
<br>
The final layer is also a fully connected layer with 2 neurins.
<br>
We have used **Relu** activation function in all the layers except the output layer in which we have used **Softmax**.

**ReLU stands for Rectified Linear Unit**: it is a linear function that will give the input as output directly if it is positive, otherwise, it will give output as zero.
**SoftMax**: it is a type of squashing function and limits the output of the function into the range 0 to 1. This allows the output to be interpreted directly as a probability.

In [5]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')])

We have used Optimizer **'Adam'**: *Adam is a replacement optimization algorithm for stochastic gradient descent for training deep learning models. Adam combines the best properties of the AdaGrad and RMSProp algorithms to provide an optimization algorithm that can handle sparse gradients on noisy problems.*
<br>
Loss funtion **'Categorical_crossentropy'**: *Categorical crossentropy is a loss function that is used in multi-class classification tasks. These are tasks where an example can only belong to one out of many possible categories, and the model must decide which one.*
<br>
We will fit and compile the model with train_batch and do the validation with test_batch.
We are saving the model as an .h5 file so that we can call the model and provide the video feed as input to the model.

In [7]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_batch, validation_data=test_batch,epochs=15,steps_per_epoch=steps_per_epoch ,validation_steps=test_steps)

model.save('models/cnndd.h5', overwrite=True)

Epoch 1/15
38/38 [==============================] - 7s 169ms/step - loss: 0.4357 - accuracy: 0.8003 - val_loss: 0.2180 - val_accuracy: 0.9323
Epoch 2/15
38/38 [==============================] - 5s 139ms/step - loss: 0.1699 - accuracy: 0.9326 - val_loss: 0.1291 - val_accuracy: 0.9531
Epoch 3/15
38/38 [==============================] - 5s 145ms/step - loss: 0.1371 - accuracy: 0.9434 - val_loss: 0.1351 - val_accuracy: 0.9688
Epoch 4/15
38/38 [==============================] - 5s 140ms/step - loss: 0.1097 - accuracy: 0.9601 - val_loss: 0.1176 - val_accuracy: 0.9531
Epoch 5/15
38/38 [==============================] - 5s 140ms/step - loss: 0.0829 - accuracy: 0.9734 - val_loss: 0.0889 - val_accuracy: 0.9583
Epoch 6/15
38/38 [==============================] - 5s 140ms/step - loss: 0.0539 - accuracy: 0.9817 - val_loss: 0.0581 - val_accuracy: 0.9792
Epoch 7/15
38/38 [==============================] - 5s 140ms/step - loss: 0.0427 - accuracy: 0.9850 - val_loss: 0.0757 - val_accuracy: 0.9688
Epoch 

We have attained an accuracy of 96.88 percent in 15 epochs.